In [33]:
#task1
from typing import List, Dict
from statistics import mean, median, multimode

def calculate_stats(numbers: List[int]) -> Dict[str, float]:
    # Расчет среднего значения, медианы и моды, обработка ситуации с несколькими модами, если это необходимо
    mean_val = mean(numbers)  # Среднее значение
    median_val = median(numbers)  # Медиана
    mode_val = multimode(numbers)  # Мода или моды
    
    # Если есть несколько мод, надо взять наименьшую
    mode_val = min(mode_val) if mode_val else None

    # Результаты округляем до 4 знаков после запятой, как показано в примере
    return {
        'mean': round(mean_val, 4), 
        'median': median_val,
        'mode': mode_val
    }

example_list = [0, 1, 1, 10, 5, 4, 3]
calculate_stats(example_list)

{'mean': 3.4286, 'median': 3, 'mode': 1}

In [34]:
#task2
def parse_csv(file_content: str) -> List[Dict]:
    # Разделение входной строки на строки таблицы
    lines = file_content.strip().split('\n')
    # Получение заголовков из первой строки
    headers = lines[0].split(',')
    # Инициализация пустого списка для хранения словарей
    records = []
    
    # Проход по оставшимся строкам, пропуская заголовок
    for line in lines[1:]:
        # Разделение строки на значения
        buff = line.split('"')
        values = buff[0].split(',')
        if values[4] == 'Zeus':
            roles = ['Nuker']
        else:
            roles = buff[1][1:-1]
            roles = roles.split(',')
        values.pop()
        values.append(roles)
        # Создание словаря для текущей строки, используя заголовки как ключи
        record = {header: value for header, value in zip(headers, values)}
        # Преобразование строки в число для ключа 'legs'
        record['legs'] = int(record['legs'])
        records.append(record)

    return records

# Чтение содержимого файла dota_hero_stats.csv
with open('/home/sensor/ml_hw/data/dota_hero_stats.csv', 'r') as file:
    content = file.read()

# Парсинг файла с учетом пустого заголовка в начале
parsed_data = parse_csv(content)

# Поиск персонажа с максимальным количеством ног
max_legs_hero = max(parsed_data, key=lambda x: int(x['legs']))
max_legs_hero

{'': '59',
 'attack_type': 'Melee',
 'id': '61',
 'legs': 8,
 'localized_name': 'Broodmother',
 'name': 'npc_dota_hero_broodmother',
 'primary_attr': 'agi',
 'roles': ["'Carry'", " 'Pusher'", " 'Escape'", " 'Nuker'"]}

In [35]:
def calculate_similarity(hero1: Dict, hero2: Dict) -> float:
    # Тип атаки: 0 если совпадает, иначе 1
    attack_type_similarity = 0 if hero1['attack_type'] == hero2['attack_type'] else 1
    
    # Количество ног: абсолютная разность
    legs_similarity = abs(int(hero1['legs']) - int(hero2['legs']))
    
    # Основной атрибут: 0 если совпадает, иначе 1
    primary_attr_similarity = 0 if hero1['primary_attr'] == hero2['primary_attr'] else 1
    
    # Роли: доля совпадающих ролей
    roles_hero1 = set(hero1['roles'])
    roles_hero2 = set(hero2['roles'])
    common_roles = roles_hero1.intersection(roles_hero2)
    total_roles = roles_hero1.union(roles_hero2)
    roles_similarity = len(common_roles) / len(total_roles) if total_roles else 1  # Если нет ролей, считаем близость полной
    
    # Инвертируем меру близости для ролей, т.к. больше общих ролей означает большую близость
    roles_similarity = 1 - roles_similarity
    
    # Суммарная мера близости
    total_similarity = attack_type_similarity + legs_similarity + primary_attr_similarity + roles_similarity
    
    return total_similarity

# Ищем двух наиболее похожих персонажей
min_similarity = float('inf')
closest_heroes_pair = None

for i in range(len(parsed_data)):
    for j in range(i+1, len(parsed_data)):
        similarity = calculate_similarity(parsed_data[i], parsed_data[j])
        if similarity < min_similarity:
            min_similarity = similarity
            closest_heroes_pair = (parsed_data[i]['localized_name'], parsed_data[j]['localized_name'])

closest_heroes_pair, min_similarity

(('Crystal Maiden', 'Keeper of the Light'), 0.0)